In [8]:
import numpy as np
from pandas_dq import dq_report
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import ElasticNetCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer
import lightgbm as lgb
import mlflow
import json
import os
from sklearn.metrics import r2_score


pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)



In [ ]:
experiment_id = mlflow.create_experiment('house-pricesTRAIN')
mlflow.set_experiment('house-pricesTRAIN')

In [69]:
housePrices = pd.read_csv('csv_files/housePricesTrain.csv')
X_train, y_train = housePrices.drop(columns=['SalePrice']), housePrices['SalePrice']
X_train = X_train.drop(columns=['Id'])

In [5]:
housePrices.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# Analysis EDA

In [7]:
X_analysis = dq_report(housePrices, verbose=2)

/var/folders/25/8m9ycrtx2kv84b8szslr4jfc0000gp/T/ipykernel_35553/2887496668.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  X_analysis = dq_report(housePrices, verbose=2)


,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
Id,int64,0.000000,100,1,1460,Possible ID colum: drop before modeling process.
MSSubClass,int64,0.000000,1,20,190,has 103 outliers greater than upper bound (145.0) or lower than lower bound(-55.0). Cap them or remove them.
MSZoning,object,0.000000,0,C (all),RM,"3 rare categories: ['FV', 'RH', 'C (all)']. Group them into a single category or drop the categories."
LotFrontage,float64,17.739726,NA,21.000000,313.000000,"259 missing values. Impute them with mean, median, mode, or a constant value such as 123., has 88 outliers greater than upper bound (111.5) or lower than lower bound(27.5). Cap them or remove them."
LotArea,int64,0.000000,73,1300,215245,has 69 outliers greater than upper bound (17673.5) or lower than lower bound(1481.5). Cap them or remove them.
Street,object,0.000000,0,Grvl,Pave,1 rare categories: ['Grvl']. Group them into a single category or drop the categories.
Alley,object,93.767123,0,nan,nan,"1369 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: float, object,"
LotShape,object,0.000000,0,IR1,Reg,"2 rare categories: ['IR2', 'IR3']. Group them into a single category or drop the categories."
LandContour,object,0.000000,0,Bnk,Lvl,"3 rare categories: ['Bnk', 'HLS', 'Low']. Group them into a single category or drop the categories."
Utilities,object,0.000000,0,AllPub,NoSeWa,1 rare categories: ['NoSeWa']. Group them into a single category or drop the categories.


# Linear Regression

I will start with a simple linear regression with ElasticNet penalty terms. I will only fill all the NaN's, one-hot encode and scale the dataset, so that the model can be fitted, but no further modifications will be pursued. This will serve as a baseline for comparison; My goal is finding out what score can I get with the least amount of effort in terms of feature engineering. There will be a little bit of hyperparameter tinkering as I will be using gridsearch.

In [70]:
X_train = X_train.fillna('None')

In [71]:
X_trainDummies = pd.get_dummies(X_train)

In [72]:
scaler = StandardScaler()
X_trainScaled = scaler.fit_transform(X_trainDummies)

In [73]:
param_grid = {
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
    'alpha': [1, 10, 100, 1000],
}

In [74]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Create a scorer object using the custom MAPE function
mape_scorer = make_scorer(mape, greater_is_better=False)

In [76]:
scoring = {
    'R-squared': make_scorer(r2_score),
    'Adjusted R-squared': make_scorer(r2_score, greater_is_better=False),
    'Mape': mape_scorer
}

In [77]:
grid = GridSearchCV(
            estimator=ElasticNet(max_iter=50000), 
            param_grid=param_grid, 
            scoring=scoring, 
            verbose = 3,
            n_jobs=-1,
            cv=5, 
            refit='Mape'
            ) 

In [78]:
grid.fit(X_trainScaled, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[CV 4/5] END alpha=1, l1_ratio=0.1; Adjusted R-squared: (test=-0.862) Mape: (test=-9.877) R-squared: (test=0.862) total time=   0.2s
[CV 2/5] END alpha=1, l1_ratio=0.1; Adjusted R-squared: (test=-0.829) Mape: (test=-11.206) R-squared: (test=0.829) total time=   0.2s
[CV 1/5] END alpha=1, l1_ratio=0.1; Adjusted R-squared: (test=-0.886) Mape: (test=-9.024) R-squared: (test=0.886) total time=   0.3s
[CV 2/5] END alpha=1, l1_ratio=0.3; Adjusted R-squared: (test=-0.831) Mape: (test=-11.101) R-squared: (test=0.831) total time=   0.3s
[CV 1/5] END alpha=1, l1_ratio=0.3; Adjusted R-squared: (test=-0.890) Mape: (test=-9.001) R-squared: (test=0.890) total time=   0.3s
[CV 3/5] END alpha=1, l1_ratio=0.1; Adjusted R-squared: (test=-0.837) Mape: (test=-10.719) R-squared: (test=0.837) total time=   0.2s
[CV 3/5] END alpha=1, l1_ratio=0.3; Adjusted R-squared: (test=-0.845) Mape: (test=-10.630) R-squared: (test=0.845) total time=   0.2s
[CV 5/5] END alpha=1, l1_ratio=0.1; Adjusted R-squared: (test=-0.

GridSearchCV(cv=5, estimator=ElasticNet(max_iter=50000), n_jobs=-1,
             param_grid={'alpha': [1, 10, 100, 1000],
                         'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]},
             refit='Mape',
             scoring={'Adjusted R-squared': make_scorer(r2_score, greater_is_better=False),
                      'Mape': make_scorer(mape, greater_is_better=False),
                      'R-squared': make_scorer(r2_score)},
             verbose=3)

In [80]:
grid_results = pd.DataFrame(grid.cv_results_)
grid_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,...,split3_test_Mape,split4_test_Mape,mean_test_Mape,std_test_Mape,rank_test_Mape
0,0.248471,0.009035,0.003597,0.001206,1,...,-9.876622,-10.693385,-10.303932,0.769304,3
1,0.264192,0.012845,0.003022,0.001591,1,...,-9.790494,-10.633197,-10.231194,0.746344,1
2,0.293548,0.026589,0.004113,0.001826,1,...,-9.830499,-10.638402,-10.241260,0.725026,2
3,0.414178,0.014495,0.004225,0.002405,1,...,-10.115113,-10.818857,-10.432815,0.706024,5
4,0.587731,0.022872,0.004619,0.002916,1,...,-11.180665,-11.507654,-11.216967,0.726505,6
...,...,...,...,...,...,...,...,...,...,...,...
15,0.021603,0.007712,0.002506,0.001338,1000,...,-36.420105,-33.992782,-35.677707,1.647451,20
16,0.019837,0.002344,0.002081,0.001243,1000,...,-36.239910,-33.828779,-35.502657,1.642273,19
17,0.023158,0.007133,0.003079,0.002842,1000,...,-35.922702,-33.538866,-35.193496,1.633474,18
18,0.032392,0.008529,0.002439,0.001786,1000,...,-35.213232,-32.886468,-34.498439,1.614001,17


In [86]:
best_Rsquared = np.amax(grid_results['mean_test_R-squared'])
best_adjustedR = np.amax(grid_results['mean_test_Adjusted R-squared'])

In [97]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)
grid_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,...,split3_test_Mape,split4_test_Mape,mean_test_Mape,std_test_Mape,rank_test_Mape
0,0.248471,0.009035,0.003597,0.001206,1,...,-9.876622,-10.693385,-10.303932,0.769304,3
1,0.264192,0.012845,0.003022,0.001591,1,...,-9.790494,-10.633197,-10.231194,0.746344,1
2,0.293548,0.026589,0.004113,0.001826,1,...,-9.830499,-10.638402,-10.241260,0.725026,2
3,0.414178,0.014495,0.004225,0.002405,1,...,-10.115113,-10.818857,-10.432815,0.706024,5
4,0.587731,0.022872,0.004619,0.002916,1,...,-11.180665,-11.507654,-11.216967,0.726505,6
...,...,...,...,...,...,...,...,...,...,...,...
15,0.021603,0.007712,0.002506,0.001338,1000,...,-36.420105,-33.992782,-35.677707,1.647451,20
16,0.019837,0.002344,0.002081,0.001243,1000,...,-36.239910,-33.828779,-35.502657,1.642273,19
17,0.023158,0.007133,0.003079,0.002842,1000,...,-35.922702,-33.538866,-35.193496,1.633474,18
18,0.032392,0.008529,0.002439,0.001786,1000,...,-35.213232,-32.886468,-34.498439,1.614001,17


Loging MLFlow Parameters.

In [54]:
mlflow.start_run(
    run_name="ElasticNet_Linear_Regression",
    experiment_id=experiment_id,
    description="Linear Regression with ElasticNet penalty",
)

<ActiveRun: >

In [39]:
with open("param_grid.json", "w") as outfile:
    json.dump(param_grid, outfile)

In [62]:
mlflow.log_artifact(local_path='/Users/matheus/Desktop/ml_project/house_prices_kaggle/json_files/param_grid.json', artifact_path='param_grid')

In [63]:
mlflow.log_param("Best MAPE", -grid.best_params_)

{'alpha': 1, 'l1_ratio': 0.3}

In [64]:
mlflow.log_metric("Best GridSearch Mape score", grid.best_score_)

# Feature Engineering

Em um primeiro momento, nós identificamos as colunas com variáveis categóricas que possuam uma ordem natural entre elas. Atribuiremos valores inteiros e ordenados a cada uma.

In [4]:
X_reduced = housePrices.drop(columns=['Id', 'SalePrice'])

In [5]:
X_reduced = X_reduced.fillna(0) #Substituiremos os valores NaN por '0', para que a ordem seja preservada 
                            # nas variáveis categóricas.

# Dicionário
mapping = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, 
           'No': 1, 'Mn': 2, 'Av': 3, 
           'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6, 
           'RFn': 2, 'Fin': 3
           }
##      
##       Ex   Excellent          GLQ   Good Living Quarters         Fin	Finished
##       Gd   Good               ALQ   Average Living Quarters      RFn	Rough Finished
##       TA   Average/Typical    Rec   Average Rec Room            
##       Fa   Fair               LwQ   Low Quality
##       Po   Poor               Unf   Unfinished
##
##

X_reduced = X_reduced.replace(mapping)

In [9]:
scaler = StandardScaler()
X_reducedScaled = scaler.fit_transform(X_reducedDummies)

In [8]:
X_reducedDummies = pd.get_dummies(X_reduced)

In [10]:
print(X_trainDummies.shape)
print(X_reducedDummies.shape)


NameError: name 'X_trainDummies' is not defined

In [14]:
gridReduced = GridSearchCV(
            estimator=ElasticNet(max_iter=50000), 
            param_grid=param_grid, 
            scoring=mape_scorer,
            verbose = 3,
            n_jobs=-1,
            cv=5, 
            return_train_score=True
            ) 

In [15]:
gridReduced.fit(X_reducedScaled, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(cv=5, estimator=ElasticNet(max_iter=50000), n_jobs=-1,
             param_grid={'alpha': [1, 10, 100, 1000],
                         'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]},
             return_train_score=True,
             scoring=make_scorer(mape, greater_is_better=False), verbose=3)

In [16]:
pd.DataFrame(gridReduced.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.035643,0.007097,0.002223,0.001094,1,0.1,"{'alpha': 1, 'l1_ratio': 0.1}",-9.567655,-11.468288,-10.480943,-9.801516,-11.248397,-10.513360,0.755552,3,-9.369586,-9.035294,-9.223534,-9.407404,-9.027400,-9.212643,0.160281
1,0.036155,0.008309,0.000951,0.000215,1,0.3,"{'alpha': 1, 'l1_ratio': 0.3}",-9.564649,-11.431530,-10.429548,-9.727452,-11.197123,-10.470060,0.751787,1,-9.180062,-8.853594,-9.008517,-9.223348,-8.836571,-9.020419,0.160274
2,0.035929,0.007356,0.001464,0.001483,1,0.5,"{'alpha': 1, 'l1_ratio': 0.5}",-9.611027,-11.440134,-10.401964,-9.826959,-11.239324,-10.503882,0.732563,2,-8.995906,-8.674069,-8.778319,-9.034303,-8.648181,-8.826156,0.160770
3,0.041854,0.007663,0.001013,0.000638,1,0.7,"{'alpha': 1, 'l1_ratio': 0.7}",-9.748460,-11.548542,-10.482078,-10.114237,-11.381839,-10.655031,0.702972,5,-8.851631,-8.557387,-8.532232,-8.827719,-8.495206,-8.652835,0.154017
4,0.056540,0.008922,0.001060,0.000518,1,0.9,"{'alpha': 1, 'l1_ratio': 0.9}",-10.271353,-11.830089,-11.013644,-10.971816,-11.911903,-11.199761,0.608769,6,-8.788366,-8.535312,-8.380225,-8.673847,-8.433924,-8.562335,0.150936
5,0.010491,0.005215,0.001136,0.000690,10,0.1,"{'alpha': 10, 'l1_ratio': 0.1}",-14.906550,-18.456451,-17.043937,-17.556443,-16.985641,-16.989804,1.167478,10,-16.793412,-16.235146,-16.762082,-16.783285,-16.503248,-16.615434,0.218431
6,0.012313,0.003953,0.001493,0.000993,10,0.3,"{'alpha': 10, 'l1_ratio': 0.3}",-13.383116,-16.734005,-15.490293,-15.876456,-15.650879,-15.426950,1.108320,9,-15.169343,-14.673135,-15.182939,-15.164958,-14.872186,-15.012512,0.205791
7,0.012223,0.003455,0.001328,0.000575,10,0.5,"{'alpha': 10, 'l1_ratio': 0.5}",-11.817665,-14.769512,-13.744605,-13.888605,-14.096933,-13.663464,0.987470,8,-13.304894,-12.869493,-13.350810,-13.333375,-13.023233,-13.176361,0.194537
8,0.012029,0.001878,0.000874,0.000546,10,0.7,"{'alpha': 10, 'l1_ratio': 0.7}",-10.383171,-12.867049,-12.020970,-11.752716,-12.479951,-11.900771,0.849544,7,-11.351772,-10.978659,-11.366191,-11.377381,-11.032063,-11.221213,0.177236
9,0.011847,0.001058,0.000571,0.000022,10,0.9,"{'alpha': 10, 'l1_ratio': 0.9}",-9.567564,-11.493145,-10.503604,-9.871690,-11.291867,-10.545574,0.757262,4,-9.471523,-9.133956,-9.337880,-9.504813,-9.123419,-9.314318,0.161573


# Kbest

In [86]:

selector = SelectKBest(score_func=f_regression, k=20)


selector.fit(X_reducedDummies, y_train)

selector.fit_transform(X_reducedDummies, y_train)

selected = selector.get_support()

In [87]:
feature_scores = dict(zip(X_reducedDummies.columns[selected], selector.scores_))

pd.DataFrame.from_dict(feature_scores, orient='index').sort_values(by=0, ascending=False)

,0
MasVnrArea,2436.770591
1stFlrSF,1272.318336
Neighborhood_NridgHt,880.341282
FullBath,759.380775
BsmtQual,548.665821
TotalBsmtSF,504.714855
HeatingQC,419.328468
Foundation_PConc,326.323107
FireplaceQu,255.923517
TotRmsAbvGrd,238.132328


In [88]:
X_Kbest = X_reducedDummies[X_reducedDummies.columns[selected]]

# Correlations

In [89]:
correlations = X_Kbest.corr()

# Step 1: Drop diagonal entries
corr_df = correlations.where(np.triu(np.ones(correlations.shape), k=1).astype(np.bool))

# Step 2: Unstack to get a Series of correlation values
corr_series = corr_df.unstack()

# Step 3: Sort the Series in descending order
corr_series_sorted = corr_series.sort_values(ascending=False)

# Print the sorted correlation values
pd.DataFrame(corr_series_sorted)

/var/folders/25/8m9ycrtx2kv84b8szslr4jfc0000gp/T/ipykernel_1247/519127393.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  corr_df = correlations.where(np.triu(np.ones(correlations.shape), k=1).astype(np.bool))


0
GarageArea           GarageCars            0.882475
FireplaceQu          Fireplaces            0.863241
TotRmsAbvGrd         GrLivArea             0.825489
1stFlrSF             TotalBsmtSF           0.819530
ExterQual            OverallQual           0.726278
KitchenQual          ExterQual             0.716122
                     OverallQual           0.673331
Foundation_PConc     YearBuilt             0.651199
FullBath             GrLivArea             0.630012
BsmtQual             OverallQual           0.629379
KitchenQual          YearRemodAdd          0.625316
Foundation_PConc     ExterQual             0.604901
BsmtQual             YearBuilt             0.602797
GarageCars           OverallQual           0.600671
ExterQual            YearBuilt             0.598160
GarageFinish         YearBuilt             0.595781
GrLivArea            OverallQual           0.593007
YearRemodAdd         YearBuilt             0.592855
ExterQual            YearRemodAdd          0.587318
GarageCars           GarageFinish          0.579729
YearBuilt            OverallQual           0.572323
Foundation_PConc     YearRemodAdd          0.569728
                     OverallQual           0.568211
GrLivArea            1stFlrSF              0.566024
BsmtQual             ExterQual             0.565558
TotalBsmtSF          BsmtQual              0.564299
GarageArea           OverallQual           0.562022
Foundation_PConc     BsmtQual              0.561151
GarageFinish         OverallQual           0.556863
TotRmsAbvGrd         FullBath              0.554784
YearRemodAdd         OverallQual           0.550684
FullBath             OverallQual           0.550600
HeatingQC            YearRemodAdd          0.550017
Foundation_PConc     KitchenQual           0.538126
GarageCars           YearBuilt             0.537850
TotalBsmtSF          OverallQual           0.537808
KitchenQual          YearBuilt             0.530165
Foundation_PConc     HeatingQC             0.529073
GarageCars           ExterQual             0.526390
HeatingQC            ExterQual             0.520027
GarageFinish         ExterQual             0.519054
GarageArea           GarageFinish          0.516428
BsmtQual             YearRemodAdd          0.515013
KitchenQual          BsmtQual              0.510938
GarageCars           KitchenQual           0.509810
KitchenQual          HeatingQC             0.504228
GarageArea           ExterQual             0.495778
FireplaceQu          OverallQual           0.490788
GarageArea           1stFlrSF              0.489782
                     KitchenQual           0.489606
                     TotalBsmtSF           0.486665
GarageFinish         BsmtQual              0.485184
FullBath             ExterQual             0.483949
GarageArea           YearBuilt             0.478954
Foundation_PConc     GarageFinish          0.477211
1stFlrSF             OverallQual           0.476224
GarageFinish         KitchenQual           0.474570
TotalBsmtSF          ExterQual             0.470580
FireplaceQu          GrLivArea             0.469947
GarageCars           FullBath              0.469672
GarageArea           GrLivArea             0.468997
FullBath             YearBuilt             0.468271
GarageCars           GrLivArea             0.467247
Fireplaces           GrLivArea             0.461679
Foundation_PConc     FullBath              0.461667
HeatingQC            OverallQual           0.457083
GrLivArea            TotalBsmtSF           0.454868
Foundation_PConc     GarageCars            0.453421
GarageCars           BsmtQual              0.449194
HeatingQC            YearBuilt             0.448855
GarageFinish         YearRemodAdd          0.444541
GarageCars           1stFlrSF              0.439317
FullBath             YearRemodAdd          0.439046
GrLivArea            ExterQual             0.435986
KitchenQual          FullBath              0.434648
GarageCars           TotalBsmtSF           0.434585
KitchenQual          TotalBsmtSF           0.4

In [90]:
X_Kbest = X_Kbest.drop(['GarageCars', 'Fireplaces', 'TotRmsAbvGrd', '1stFlrSF'], axis=1)

In [91]:
X_Kbest.shape

(1460, 16)

# Kbest fit

In [92]:
KbestGrid = GridSearchCV(
            estimator=ElasticNet(max_iter=50000), 
            param_grid=param_grid, 
            scoring=mape_scorer,
            verbose = 3,
            n_jobs=-1,
            cv=5, 
            return_train_score=True
            ) 

In [93]:
KbestGrid.fit(X_Kbest, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(cv=5, estimator=ElasticNet(max_iter=50000), n_jobs=-1,
             param_grid={'alpha': [1, 10, 100, 1000],
                         'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]},
             return_train_score=True,
             scoring=make_scorer(mape, greater_is_better=False), verbose=3)

In [94]:
pd.DataFrame(KbestGrid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.005921,0.001627,0.002435,0.000504,1,0.1,"{'alpha': 1, 'l1_ratio': 0.1}",-13.023353,-13.649039,-12.269914,-13.020097,-15.587100,-13.509901,1.126794,4,-13.400423,-13.343476,-13.140602,-13.484153,-13.321046,-13.337940,0.113614
1,0.005880,0.001932,0.002242,0.000778,1,0.3,"{'alpha': 1, 'l1_ratio': 0.3}",-12.951190,-13.714716,-12.252532,-12.957313,-15.436279,-13.462406,1.089963,3,-13.378741,-13.278615,-13.110733,-13.451089,-13.261386,-13.296113,0.115519
2,0.007366,0.005008,0.003553,0.002038,1,0.5,"{'alpha': 1, 'l1_ratio': 0.5}",-12.888089,-13.832717,-12.256793,-12.926962,-15.255410,-13.431994,1.040904,1,-13.383845,-13.239589,-13.093459,-13.434122,-13.203035,-13.270810,0.123668
3,0.005411,0.001761,0.003184,0.001980,1,0.7,"{'alpha': 1, 'l1_ratio': 0.7}",-12.899518,-14.050824,-12.310960,-12.939695,-15.032602,-13.446720,0.972257,2,-13.453819,-13.253235,-13.126309,-13.469161,-13.157790,-13.292063,0.144596
4,0.006914,0.004984,0.001966,0.000988,1,0.9,"{'alpha': 1, 'l1_ratio': 0.9}",-13.066916,-14.408126,-12.470352,-13.143755,-14.764634,-13.570756,0.868747,6,-13.641042,-13.413042,-13.293221,-13.620714,-13.160856,-13.425775,0.185610
5,0.002060,0.000673,0.000929,0.000035,10,0.1,"{'alpha': 10, 'l1_ratio': 0.1}",-14.113335,-13.936216,-13.065096,-14.173769,-17.001914,-14.458066,1.332931,10,-14.283502,-14.411961,-14.116729,-14.373854,-14.144086,-14.266026,0.118649
6,0.002395,0.000977,0.001152,0.000219,10,0.3,"{'alpha': 10, 'l1_ratio': 0.3}",-14.010764,-13.857496,-12.965851,-14.071315,-16.896304,-14.360346,1.329233,9,-14.185996,-14.315376,-14.013999,-14.283569,-14.065357,-14.172859,0.117927
7,0.002840,0.000886,0.001167,0.000228,10,0.5,"{'alpha': 10, 'l1_ratio': 0.5}",-13.850608,-13.748473,-12.821551,-13.913820,-16.733420,-14.213575,1.321005,8,-14.042399,-14.167676,-13.860463,-14.144900,-13.941223,-14.031332,0.117410
8,0.001863,0.000205,0.001196,0.000568,10,0.7,"{'alpha': 10, 'l1_ratio': 0.7}",-13.575869,-13.591326,-12.607682,-13.632576,-16.439947,-13.969480,1.293764,7,-13.806201,-13.907889,-13.607321,-13.915196,-13.740510,-13.795424,0.114463
9,0.002553,0.001074,0.001233,0.000522,10,0.9,"{'alpha': 10, 'l1_ratio': 0.9}",-13.062032,-13.622125,-12.286543,-13.056637,-15.657963,-13.537060,1.142429,5,-13.417338,-13.380990,-13.160216,-13.505392,-13.349836,-13.362754,0.113882




# Polynomial Regression

In [98]:
poly_reg = make_pipeline(PolynomialFeatures(2), 
                         GridSearchCV(
                            estimator=ElasticNet(max_iter=50000), 
                            param_grid=param_grid, 
                            scoring=mape_scorer,
                            verbose = 3,
                            n_jobs=-1,
                            cv=5, 
                            return_train_score=True) )

In [99]:
poly_reg.fit(X_Kbest, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.549e+11, tolerance: 7.706e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.185e+11, tolerance: 7.249e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features

/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.118e+11, tolerance: 6.990e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.470e+11, tolerance: 7.706e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features

/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.487e+11, tolerance: 7.592e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.515e+11, tolerance: 7.288e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features

/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.250e+11, tolerance: 7.249e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.618e+11, tolerance: 7.706e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features

/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.322e+11, tolerance: 7.249e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.282e+11, tolerance: 6.990e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('gridsearchcv',
                 GridSearchCV(cv=5, estimator=ElasticNet(max_iter=50000),
                              n_jobs=-1,
                              param_grid={'alpha': [1, 10, 100, 1000],
                                          'l1_ratio': [0.1, 0.3, 0.5, 0.7,
                                                       0.9]},
                              return_train_score=True,
                              scoring=make_scorer(mape, greater_is_better=False),
                              verbose=3))])

In [100]:
estimator = poly_reg.named_steps['gridsearchcv']

In [101]:
pd.DataFrame(estimator.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,8.616657,0.075487,0.002367,0.000835,1,0.1,"{'alpha': 1, 'l1_ratio': 0.1}",-11.539073,-13.541788,-10.558826,-11.893072,-12.644073,-12.035366,1.008618,16,-10.800442,-10.376769,-10.975236,-10.668716,-10.731084,-10.710449,0.195817
1,8.927579,0.371469,0.001398,0.000455,1,0.3,"{'alpha': 1, 'l1_ratio': 0.3}",-11.553714,-13.548399,-10.551136,-11.883412,-12.660216,-12.039376,1.013671,17,-10.782828,-10.364381,-10.957610,-10.652338,-10.717912,-10.695014,0.194078
2,9.429297,0.107922,0.001076,0.000631,1,0.5,"{'alpha': 1, 'l1_ratio': 0.5}",-11.576166,-13.561345,-10.541286,-11.874379,-12.682305,-12.047096,1.021324,18,-10.757903,-10.346505,-10.935124,-10.632787,-10.701789,-10.674822,0.192326
3,9.645358,0.136077,0.001001,0.000470,1,0.7,"{'alpha': 1, 'l1_ratio': 0.7}",-11.613096,-13.594439,-10.525620,-11.862757,-12.717600,-12.062702,1.037290,19,-10.724695,-10.315571,-10.904805,-10.608932,-10.677494,-10.646299,0.192209
4,9.831820,0.237723,0.000783,0.000265,1,0.9,"{'alpha': 1, 'l1_ratio': 0.9}",-11.716424,-13.711789,-10.510414,-11.820287,-12.783743,-12.108532,1.078625,20,-10.668223,-10.245918,-10.854641,-10.569986,-10.623651,-10.592484,0.198014
5,10.324988,0.051931,0.000832,0.000351,10,0.1,"{'alpha': 10, 'l1_ratio': 0.1}",-11.441692,-13.512897,-10.585679,-11.958543,-12.522379,-12.004238,0.987459,11,-10.930958,-10.442038,-11.093992,-10.782673,-10.823859,-10.814704,0.215177
6,10.643395,0.232741,0.000716,0.000062,10,0.3,"{'alpha': 10, 'l1_ratio': 0.3}",-11.448283,-13.519663,-10.585844,-11.956383,-12.535531,-12.009141,0.990140,12,-10.921496,-10.437588,-11.085698,-10.776022,-10.816794,-10.807519,0.213683
7,10.808195,0.124639,0.001070,0.000439,10,0.5,"{'alpha': 10, 'l1_ratio': 0.5}",-11.458610,-13.524472,-10.584326,-11.952969,-12.552467,-12.014569,0.992736,13,-10.906376,-10.431000,-11.072838,-10.764944,-10.806546,-10.796341,0.211245
8,11.102936,0.025508,0.001041,0.000371,10,0.7,"{'alpha': 10, 'l1_ratio': 0.7}",-11.477757,-13.529509,-10.583183,-11.942402,-12.576203,-12.021811,0.995234,14,-10.878787,-10.418990,-11.049235,-10.742831,-10.788949,-10.775759,0.206856
9,11.247253,0.134148,0.001037,0.000317,10,0.9,"{'alpha': 10, 'l1_ratio': 0.9}",-11.531116,-13.539067,-10.562710,-11.898551,-12.635355,-12.033360,1.006259,15,-10.809394,-10.382846,-10.984060,-10.676906,-10.738206,-10.718283,0.196756


In [102]:
deg3_regression = make_pipeline(PolynomialFeatures(3), 
                         GridSearchCV(
                            estimator=ElasticNet(max_iter=50000), 
                            param_grid=param_grid, 
                            scoring=mape_scorer,
                            verbose = 3,
                            n_jobs=-1,
                            cv=5, 
                            return_train_score=True) )

In [103]:
deg3_regression.fit(X_Kbest, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.669e+11, tolerance: 6.990e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.654e+11, tolerance: 7.249e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features

/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.817e+11, tolerance: 7.592e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.823e+11, tolerance: 7.592e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features

/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.681e+11, tolerance: 6.990e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.906e+11, tolerance: 7.592e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features

/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.802e+11, tolerance: 6.990e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.711e+11, tolerance: 7.288e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features

/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.013e+11, tolerance: 7.706e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.988e+11, tolerance: 6.990e+08
  model = cd_fast.enet_coordinate_descent(
/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('gridsearchcv',
                 GridSearchCV(cv=5, estimator=ElasticNet(max_iter=50000),
                              n_jobs=-1,
                              param_grid={'alpha': [1, 10, 100, 1000],
                                          'l1_ratio': [0.1, 0.3, 0.5, 0.7,
                                                       0.9]},
                              return_train_score=True,
                              scoring=make_scorer(mape, greater_is_better=False),
                              verbose=3))])

In [104]:
estimator3 = deg3_regression.named_steps['gridsearchcv']

In [105]:
pd.DataFrame(estimator3.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,119.245909,0.817886,0.005739,0.001295,1,0.1,"{'alpha': 1, 'l1_ratio': 0.1}",-15.852641,-20.246451,-14.183847,-14.085771,-20.705553,-17.014853,2.898598,13,-8.015998,-7.817326,-8.166291,-7.935372,-7.931172,-7.973232,0.115421
1,119.573534,1.511087,0.004978,0.002753,1,0.3,"{'alpha': 1, 'l1_ratio': 0.3}",-15.834186,-20.315283,-14.218225,-14.087934,-20.744088,-17.039943,2.918179,15,-8.000798,-7.800663,-8.145938,-7.919393,-7.911093,-7.955577,0.114515
2,121.246276,0.781516,0.004141,0.002530,1,0.5,"{'alpha': 1, 'l1_ratio': 0.5}",-15.802286,-20.392491,-14.259915,-14.097004,-20.803399,-17.071019,2.943453,18,-7.976261,-7.775750,-8.115157,-7.895327,-7.879040,-7.928307,0.113138
3,121.570598,0.930087,0.002207,0.000304,1,0.7,"{'alpha': 1, 'l1_ratio': 0.7}",-15.735707,-20.458370,-14.336328,-14.121786,-20.885917,-17.107622,2.965821,19,-7.932110,-7.729913,-8.059370,-7.853267,-7.819692,-7.878870,0.111140
4,122.162082,0.281200,0.003766,0.003163,1,0.9,"{'alpha': 1, 'l1_ratio': 0.9}",-15.544198,-20.407349,-14.499194,-14.082426,-21.126250,-17.131883,3.014453,20,-7.794084,-7.600945,-7.882213,-7.739627,-7.666456,-7.736665,0.097815
5,121.492992,1.277875,0.007527,0.003179,10,0.1,"{'alpha': 10, 'l1_ratio': 0.1}",-15.997110,-19.698890,-14.049144,-14.003595,-20.910291,-16.931806,2.872035,9,-8.113827,-7.925060,-8.278329,-8.068356,-8.034805,-8.084075,0.115395
6,120.021722,1.828530,0.003483,0.001361,10,0.3,"{'alpha': 10, 'l1_ratio': 0.3}",-15.992162,-19.732654,-14.045262,-14.009243,-20.862346,-16.928333,2.865319,8,-8.105092,-7.913676,-8.272522,-8.056145,-8.026419,-8.074771,0.117184
7,119.548235,1.129746,0.002244,0.000290,10,0.5,"{'alpha': 10, 'l1_ratio': 0.5}",-15.975141,-19.780704,-14.043404,-14.013892,-20.799681,-16.922564,2.858270,5,-8.092505,-7.898723,-8.262863,-8.038553,-8.016089,-8.061747,0.118822
8,121.913616,1.883576,0.002316,0.000439,10,0.7,"{'alpha': 10, 'l1_ratio': 0.7}",-15.939169,-19.876448,-14.057720,-14.040798,-20.709773,-16.924782,2.847713,7,-8.071820,-7.878328,-8.243496,-8.010794,-7.998550,-8.040598,0.119273
9,124.126185,1.968895,0.002734,0.001084,10,0.9,"{'alpha': 10, 'l1_ratio': 0.9}",-15.861617,-20.205530,-14.162996,-14.079960,-20.693132,-17.000647,2.890875,12,-8.024213,-7.826356,-8.177927,-7.944870,-7.942245,-7.983122,0.116058


[CV 2/5] END alpha=1, l1_ratio=0.1;, score=(train=-11.149, test=-11.962) total time=  15.0s
[CV 5/5] END alpha=1, l1_ratio=0.5;, score=(train=-10.571, test=-12.818) total time=  15.9s
[CV 5/5] END alpha=1, l1_ratio=0.7;, score=(train=-10.252, test=-12.444) total time=  17.4s
[CV 2/5] END alpha=10, l1_ratio=0.1;, score=(train=-13.184, test=-13.342) total time=  19.5s
[CV 3/5] END alpha=10, l1_ratio=0.5;, score=(train=-12.385, test=-11.737) total time=  20.3s
[CV 1/5] END alpha=10, l1_ratio=0.9;, score=(train=-11.327, test=-11.373) total time=  20.7s
[CV 4/5] END alpha=100, l1_ratio=0.1;, score=(train=-13.674, test=-13.910) total time=   2.3s
[CV 1/5] END alpha=100, l1_ratio=0.5;, score=(train=-13.684, test=-13.681) total time=   2.4s
[CV 2/5] END alpha=100, l1_ratio=0.7;, score=(train=-13.643, test=-13.801) total time=   3.2s
[CV 3/5] END alpha=100, l1_ratio=0.9;, score=(train=-12.893, test=-12.154) total time=   4.1s
[CV 5/5] END alpha=1, l1_ratio=0.1;, score=(train=-13.321, test=-15.5

[CV 3/5] END alpha=1, l1_ratio=0.3;, score=(train=-10.845, test=-10.559) total time=  15.0s
[CV 1/5] END alpha=1, l1_ratio=0.7;, score=(train=-10.728, test=-10.560) total time=  16.4s
[CV 4/5] END alpha=1, l1_ratio=0.9;, score=(train=-10.522, test=-10.811) total time=  17.7s
[CV 2/5] END alpha=10, l1_ratio=0.3;, score=(train=-13.023, test=-13.204) total time=  19.3s
[CV 5/5] END alpha=10, l1_ratio=0.5;, score=(train=-12.514, test=-15.177) total time=  19.8s
[CV 3/5] END alpha=10, l1_ratio=0.9;, score=(train=-11.045, test=-10.695) total time=  20.9s
[CV 1/5] END alpha=100, l1_ratio=0.3;, score=(train=-13.705, test=-13.640) total time=   2.4s
[CV 2/5] END alpha=100, l1_ratio=0.5;, score=(train=-13.704, test=-13.892) total time=   2.5s
[CV 3/5] END alpha=100, l1_ratio=0.7;, score=(train=-13.334, test=-12.477) total time=   2.7s
[CV 4/5] END alpha=100, l1_ratio=0.9;, score=(train=-13.221, test=-13.327) total time=   4.4s
[CV 4/5] END alpha=1, l1_ratio=0.1;, score=(train=-13.484, test=-13.0

[CV 4/5] END alpha=1, l1_ratio=0.1;, score=(train=-11.342, test=-11.284) total time=  14.8s
[CV 2/5] END alpha=1, l1_ratio=0.5;, score=(train=-10.768, test=-11.873) total time=  16.4s
[CV 3/5] END alpha=1, l1_ratio=0.9;, score=(train=-10.253, test=-10.600) total time=  17.9s
[CV 1/5] END alpha=10, l1_ratio=0.3;, score=(train=-12.988, test=-13.056) total time=  18.3s
[CV 1/5] END alpha=10, l1_ratio=0.5;, score=(train=-12.735, test=-12.804) total time=  19.9s
[CV 4/5] END alpha=10, l1_ratio=0.7;, score=(train=-12.314, test=-12.326) total time=  20.7s
[CV 3/5] END alpha=100, l1_ratio=0.1;, score=(train=-13.463, test=-12.584) total time=   2.3s
[CV 4/5] END alpha=100, l1_ratio=0.3;, score=(train=-13.665, test=-13.893) total time=   2.0s
[CV 4/5] END alpha=100, l1_ratio=0.5;, score=(train=-13.653, test=-13.859) total time=   2.3s
[CV 1/5] END alpha=100, l1_ratio=0.9;, score=(train=-13.220, test=-13.299) total time=   3.6s
[CV 5/5] END alpha=1000, l1_ratio=0.1;, score=(train=-14.207, test=-1

[CV 1/5] END alpha=1, l1_ratio=0.3;, score=(train=-11.091, test=-11.074) total time=  14.7s
[CV 1/5] END alpha=1, l1_ratio=0.5;, score=(train=-10.921, test=-10.843) total time=  16.4s
[CV 2/5] END alpha=1, l1_ratio=0.9;, score=(train=-10.238, test=-11.770) total time=  17.8s
[CV 4/5] END alpha=10, l1_ratio=0.1;, score=(train=-13.148, test=-13.246) total time=  19.4s
[CV 4/5] END alpha=10, l1_ratio=0.5;, score=(train=-12.739, test=-12.807) total time=  19.9s
[CV 2/5] END alpha=10, l1_ratio=0.9;, score=(train=-11.239, test=-11.997) total time=  20.7s
[CV 5/5] END alpha=100, l1_ratio=0.1;, score=(train=-13.366, test=-16.135) total time=   2.0s
[CV 5/5] END alpha=100, l1_ratio=0.3;, score=(train=-13.347, test=-16.116) total time=   1.9s
[CV 1/5] END alpha=100, l1_ratio=0.7;, score=(train=-13.612, test=-13.667) total time=   2.6s
[CV 2/5] END alpha=100, l1_ratio=0.9;, score=(train=-13.259, test=-13.405) total time=   4.3s
[CV 5/5] END alpha=1000, l1_ratio=0.7;, score=(train=-13.575, test=-1

# Prediction

In [112]:
X_test = pd.read_csv('test.csv')


In [130]:
X_train.shape

(1460, 837)

In [114]:
poly_reg.named_steps['gridsearchcv'].predict(X_test)

/Users/matheus/Desktop/ml_project/env/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but ElasticNet was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: 'RH'

In [115]:
test_reduced = X_test.drop(columns=['Id'])

In [132]:
columns_df1 = set(X_reducedDummies.columns)
columns_df2 = set(test_reducedDummies.columns)

different_columns = columns_df1.symmetric_difference(columns_df2)

print("Columns different between df1 and df2:", different_columns)

Columns different between df1 and df2: {'Functional_0', 'HouseStyle_2.5Fin', 'RoofMatl_Roll', 'Condition2_RRAn', 'Heating_Floor', 'RoofMatl_Metal', 'RoofMatl_Membran', 'Exterior1st_Stone', 'SaleType_0', 'Exterior2nd_Other', 'Electrical_0', 'RoofMatl_ClyTile', 'MiscFeature_TenC', 'Heating_OthW', 'Condition2_RRAe', 'Utilities_NoSeWa', 'MSZoning_0', 'Electrical_Mix', 'Exterior2nd_0', 'Exterior1st_ImStucc', 'Condition2_RRNn', 'Utilities_0', 'Exterior1st_0'}


In [137]:
test_Kbest = X_reducedDummies.columns[selected]
test_Kbest = test_reducedDummies[test_Kbest]

In [144]:

test_Kbest = test_Kbest.drop(['GarageCars', 'Fireplaces', 'TotRmsAbvGrd', '1stFlrSF'], axis=1)

In [148]:
final_prediction = poly_reg.named_steps['gridsearchcv'].predict(test_poly)

In [146]:
test_poly = poly_reg.named_steps['polynomialfeatures'].transform(test_Kbest)

# LightGBM

In [17]:
gbX_train, gbY_train = housePrices.drop(columns=['SalePrice']), housePrices['SalePrice']

In [18]:
gbX_reduced = housePrices.drop(columns=['Id', 'SalePrice'])

gbX_reduced = gbX_reduced.fillna(0) #Substituiremos os valores NaN por '0', para que a ordem seja preservada 
                                    # nas variáveis categóricas.

# Dicionário
mapping = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, 
           'No': 1, 'Mn': 2, 'Av': 3, 
           'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6, 
           'RFn': 2, 'Fin': 3
           }
##      
##       Ex   Excellent          GLQ   Good Living Quarters         Fin	Finished
##       Gd   Good               ALQ   Average Living Quarters      RFn	Rough Finished
##       TA   Average/Typical    Rec   Average Rec Room            
##       Fa   Fair               LwQ   Low Quality
##       Po   Poor               Unf   Unfinished
##
##

gbX_reduced = gbX_reduced.replace(mapping)

gbX_reducedDummies = pd.get_dummies(gbX_reduced)

In [ ]:
params = {
    'objective': 'multiclass',
    'num_class': 7,
    'metric': 'multi_logloss'
}

train_data = lgb.Dataset(gbX_reducedDummies, label=gbY_train)

# Train the LightGBM model
model = lgb.train(params, train_data)

In [ ]:
lgb_regressor = lgb.LGBMRegressor()

# Define the parameter grid for the grid search
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 5, 7, 10, 15]
}

# Perform grid search with cross-validation
gbm_search = GridSearchCV(estimator=lgb_regressor, 
                          param_grid=param_grid, 
                          cv=5, 
                          scoring='accuracy')

gbm_search.fit(gbX_reducedDummies, gbY_train)

# Get the best parameters and best score
best_params = gbm_search.best_params_
best_score = gbm_search.best_score_


MLFLOW

In [2]:
import mlflow

mlflow.set_experiment("House_Prices_Train")
with mlflow.start_run():
    run = mlflow.active_run()
    run_id = run.info.run_id

2023/07/10 00:23:49 INFO mlflow.tracking.fluent: Experiment with name 'House_Prices_Train' does not exist. Creating a new experiment.
